# Анализ музыкальных предпочтений

Цель: Провести сравнение музыкальных предпочтений у пользователей Яндекс.Музыки, проживающих в Москве и Санкт-Петербурге.

Данные предоставлены сервисом Яндекс.Музыка

# Содержание

1. Получение данных
2. Предобработка данных
3. Проверка гипотез
4. Результаты исследования

## Получение данных

 Изучим данные, предоставленные сервисом для проекта.

Импорт библиотек

In [ ]:
import pandas as pd

Прочитаем файл music_project.csv и сохраним его в переменной df.

In [ ]:
df = pd.read_csv('music_project.csv')

Получение первых 10 строк таблицы.

In [ ]:
df.head(10)

Общая информация о данных таблицы df.

In [ ]:
df.info()

Рассмотрим полученную информацию подробнее.

Всего в таблице 7 столбцов, тип данных у каждого столбца - object.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

Выводы:

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы time, day и City. Данные из столбца genre позволят узнать самые популярные жанры.

## Предобработка данных
Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

In [ ]:
df.columns

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.

In [ ]:
df = df.rename(
    columns={
        '  userID': 'user_id',
        'Track': 'track',
        '  City  ': 'city',
        'Day': 'day'
    }
)

In [ ]:
df.columns

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [ ]:
df.isna().sum()

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека, исполнителя и жанра на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']
for item in columns_to_replace:
    df[item] = df[item].fillna('unknown')

In [ ]:
df.isna().sum()

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

Теперь избавляемся от неявных дубликатов в колонке genre. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Соблюдая алфавитный порядок, сохраняем список уникальных значений столбца с жанрами в переменной unique_genres.

In [ ]:
unique_genres = df['genre'].sort_values()
unique_genres = unique_genres.unique()
unique_genres

Просматриваем список и находим неявные дубликаты названия hiphop:

hip,
hop,
hip-hop.

Чтобы очистить от них таблицу, используем метод replace().

In [ ]:
df = df.replace('hip', 'hiphop')
df = df.replace('hop', 'hiphop')
df = df.replace('hip-hop', 'hiphop')

Проверим, что заменили неправильные названия:

In [ ]:
unique_genres = df['genre'].sort_values()
unique_genres = unique_genres.unique()
unique_genres

Вывод

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, а заполним пропущенные имена исполнителей и названия треков (хоть эти пропуски и могут повлиять на результаты исследования). Имена столбцов теперь корректны и удобны для дальнейшей работы.

## Проверка гипотез

Действительно ли музыку в разных городах слушают по-разному?
Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных в эти дни композиций с известным жанром, и сравниваем результаты.

Группируем данные по городу и вызовом метода count() подсчитываем композиции, для которых известен жанр.

In [ ]:
df.groupby('city')['city'].count()

В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [ ]:
df.groupby('day')['day'].count()

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [ ]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [ ]:
number_tracks('Monday', 'Moscow')

In [ ]:
number_tracks('Monday', 'Saint-Petersburg')

In [ ]:
number_tracks('Wednesday', 'Moscow')

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg')

In [ ]:
number_tracks('Friday', 'Moscow')

In [ ]:
number_tracks('Friday', 'Saint-Petersburg')

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] - названия столбцов.

In [ ]:
info = pd.DataFrame(data= [['Moscow', 15740, 11056, 15945], ['Saint-Petersburg', 5614, 7003, 5895]] , columns=['city', 'monday', 'wednesday', 'friday'])
info

Вывод:

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

Утро понедельника и вечер пятницы — разная музыка или одна и та же?
Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получим таблицы данных по Москве moscow_general и по Санкт-Петербургу spb_general.

In [ ]:
moscow_general = df[df['city'] == 'Moscow']

In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаём функцию genre_weekday(), которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [ ]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10]
    

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр world.

В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр world.

Вывод:

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: 
для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени. 
Вторая гипотеза подтвердилась, но лишь частично.

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

Сгруппируем таблицу moscow_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице moscow_genres.

Просмотрим первые 10 строк этой новой таблицы.

In [ ]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [ ]:
moscow_genres.head(10)

Сгруппируем таблицу spb_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице spb_genres.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [ ]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [ ]:
spb_genres.head(10)

Выводы

Гипотеза частично подтвердилась:

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

## Результаты исследования

Рабочие гипотезы:
   1. Музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме.

Полностью подтверждена.
   
   2. Списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия.

В Москве слушают музыку жанра “world”,
В Петербурге — джаз и классику.
Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.
      
   3. Население двух городов предпочитает разные музыкальные жанры.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.